[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20DistilBERT.ipynb)

## Import DistilBERT models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for DistilBERT from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use DistilBERT models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) model from HuggingFace as an example
- In addition to `TFDistilBertModel` we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertModel
import tensorflow as tf

MODEL_NAME = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME).save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFDistilBertModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFDistilBertModel.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")       
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 518704
-rw-r--r--  1 maziyar  staff        518 Dec 15 14:46 config.json
drwxr-xr-x  3 maziyar  staff         96 Dec 15 14:46 saved_model
-rw-r--r--  1 maziyar  staff  265571968 Dec 15 14:46 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 9472
drwxr-xr-x  2 maziyar  staff       64 Dec 15 14:46 assets
-rw-r--r--  1 maziyar  staff       55 Dec 15 14:46 fingerprint.pb
-rw-r--r--  1 maziyar  staff    77329 Dec 15 14:46 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  4764278 Dec 15 14:46 saved_model.pb
drwxr-xr-x  4 maziyar  staff      128 Dec 15 14:46 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 472
-rw-r--r--  1 maziyar  staff     125 Dec 15 14:46 special_tokens_map.json
-rw-r--r--  1 maziyar  staff     412 Dec 15 14:46 tokenizer_config.json
-rw-r--r--  1 maziyar  staff  231508 Dec 15 14:46 vocab.txt


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.txt` from the tokenizer
- all we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for

In [6]:
!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets

## Import and Save DistilBERT in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [7]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.1 and Spark NLP 4.2.4
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.4


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `DistilBertEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively..


In [9]:
from sparknlp.annotator import *

distil_bert = DistilBertEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)\
 .setDimension(768)\
 .setStorageRef('distilbert_base_uncased') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
distil_bert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DistilERT model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀 

In [13]:
! ls -l {MODEL_NAME}_spark_nlp

total 527720
-rw-r--r--  1 maziyar  staff  270191794 Dec 15 14:53 distilbert_tensorflow
drwxr-xr-x  4 maziyar  staff        128 Dec 15 14:53 fields
drwxr-xr-x  6 maziyar  staff        192 Dec 15 14:53 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DistilBERT model 😊 

In [14]:
distilbert_loaded = DistilBertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

In [15]:
distilbert_loaded.getStorageRef()

'distilbert_base_uncased'

: 

That's it! You can now go wild and use hundreds of DistilBERT models from HuggingFace 🤗 in Spark NLP 🚀 
